In [24]:
import os
os.environ['TF_USE_LEGACY_KERAS'] = '1'

In [10]:
import tensorflow_hub as hub
import tensorflow_text as text
import pandas as pd
from sklearn.utils import resample
import tensorflow as tf

In [2]:
df = pd.read_csv("/content/spam_or_not_spam.csv")
df.head(5)

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [3]:
df['label'].value_counts()

,count
label,
0,2500
1,500


In [4]:
df_minority = df[df.label==1]
df_majority = df[df.label==0]

df_minority_upsampled = resample(
                            df_minority,
                            replace = True,
                            n_samples=len(df_majority),
                            random_state=42

)
df_balanced = pd.concat([df_majority, df_minority_upsampled])
df_balanced.head()

,email,label
0,date wed NUMBER aug NUMBER NUMBER NUMBER NUMB...,0
1,martin a posted tassos papadopoulos the greek ...,0
2,man threatens explosion in moscow thursday aug...,0
3,klez the virus that won t die already the most...,0
4,in adding cream to spaghetti carbonara which ...,0


In [5]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df_balanced['email'],df_balanced['label'], stratify=df_balanced['label'])
X_train.head(10)

,email
2809,take control of your computer with this top o...
2716,over NUMBER NUMBER the first year most of tha...
1484,jm URL justin mason writes date_in_past_NUMBER...
2577,drywall your home refinance loan is approved ...
2622,do you like sexy animals doing the wild thing...
2977,here is an excerpt from your local newspaper a...
2509,dear ricardoNUMBER cost effective direct email...
1771,use perl daily headline mailer perl ports page...
2723,dear sir my name is dr steven m duba the son o...
2486,here for your enjoyment is a little something ...


In [6]:
y_test.value_counts()

,count
label,
0,625
1,625


In [ ]:
# Install transformers
!pip install transformers -q

from transformers import TFBertModel, BertTokenizer
import tensorflow as tf

# Load BERT model (6-layer version to match your original)
bert = TFBertModel.from_pretrained('google/bert_uncased_L-6_H-768_A-12')
tokenizer = BertTokenizer.from_pretrained('google/bert_uncased_L-6_H-768_A-12')

# Build the classifier model
input_ids = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='input_ids')
attention_mask = tf.keras.layers.Input(shape=(128,), dtype=tf.int32, name='attention_mask')

# BERT encoding
bert_output = bert(input_ids, attention_mask=attention_mask)
pooled_output = bert_output.pooler_output  # This is equivalent to pooled_output from TF Hub

# Classification layers
dropout = tf.keras.layers.Dropout(0.2)(pooled_output)
output = tf.keras.layers.Dense(1, activation='sigmoid', name='classifier')(dropout)

# Create model
model = tf.keras.Model(inputs=[input_ids, attention_mask], outputs=output)

print("✅ Model created successfully!")
model.summary()

# Compile the model
model.compile(
    optimizer=tf.keras.optimizers.Adam(learning_rate=2e-5),
    loss='binary_crossentropy',
    metrics=['accuracy']
)

# Helper function to prepare your text data
def prepare_data(texts, labels=None, max_length=128):
    """
    Converts text to BERT input format

    Args:
        texts: List of strings
        labels: List of labels (optional)
        max_length: Maximum sequence length

    Returns:
        If labels provided: ((input_ids, attention_mask), labels)
        If no labels: (input_ids, attention_mask)
    """
    encoded = tokenizer(
        texts,
        max_length=max_length,
        padding='max_length',
        truncation=True,
        return_tensors='np'
    )

    if labels is not None:
        import numpy as np
        return (encoded['input_ids'], encoded['attention_mask']), np.array(labels)

    return encoded['input_ids'], encoded['attention_mask']

# Example usage:
print("\n📝 Example usage:")
print("texts = ['This movie is great!', 'This movie is terrible!']")
print("labels = [1, 0]")
print("X, y = prepare_data(texts, labels)")
print("model.fit(X, y, epochs=3, batch_size=2)")